In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from natsort import natsorted
from collections import defaultdict
import re

In [89]:
class XdcParser:
    def __init__(self, fName):
        '''
        fName is a Xilinx Master .xdc file from their website
        '''
        re_key = r'set_property\s(\w*)\s"?(\w*)"?\s\[get_ports\s"?([\w\[\]]*)"?\]'
        re_keys = r'set_property\s-dict\s\{([\w\s"]*)\}\s\[get_ports\s"?([\w\[\]]*)"?\]'
        
        with open(fName) as f:
            lines = f.readlines()

#         lines = (
#             'set_property IOSTANDARD LVCMOS18 [get_ports USB_SMSC_NXT]',
#             'set_property PACKAGE_PIN R5 [get_ports "DDR_RAS_n"]',
#             'set_property PIO_DIRECTION "BIDIR" [get_ports "MIO[53]"]',
#             'set_property -dict {PACKAGE_PIN A8 IOSTANDARD DIFF_HSTL_II_25} [get_ports FMC1_LA_24_N]'
#         )
            
        self.props = defaultdict(dict)
        for line in lines:
            line = line.strip()
            if line.startswith('#'):
                continue
            dat = re.findall(re_key, line)
            if len(dat) == 1:
                dat_key, dat_value, dat_group = dat[0]
                self.props[dat_group][dat_key] = dat_value
                continue
                
            dat = re.findall(re_keys, line)
            if len(dat) == 1:
                dat_items, dat_group = dat[0]
                dat_items = dat_items.split()
                for (k, v) in zip(dat_items[0::2], dat_items[1::2]):
                    self.props[dat_group][k] = v.replace('"', '')
                continue
                
            else:
                print('sorry, parser is too dumb for this:\n', line)

In [77]:
re_key = r'set_property\s(\w*)\s"?(\w*)"?\s\[get_ports\s"?([\w\[\]]*)"?\]'
re_keys = r'set_property\s-dict\s\{([\w\s]*)\}\s\[get_ports\s"?([\w\[\]]*)"?\]'

In [80]:
lines = (
    'set_property IOSTANDARD LVCMOS18 [get_ports USB_SMSC_NXT]',
    'set_property PACKAGE_PIN R5 [get_ports "DDR_RAS_n"]',
    'set_property PIO_DIRECTION "BIDIR" [get_ports "MIO[53]"]',
    'set_property -dict {PACKAGE_PIN A8 IOSTANDARD DIFF_HSTL_II_25} [get_ports FMC1_LA_24_N]'
)

In [82]:
for line in lines:
    dat = re.findall(re_key, line)
    if len(dat) == 1:
        dat_key, dat_value, dat_group = dat[0]
        print('single assignment', dat_group, {dat_key: dat_value})
        continue
    dat = re.findall(re_keys, line)
    if len(dat) == 1:
        dat_items, dat_group = dat[0]
        dat_items = dat_items.split()
        dat_items = {k: v for (k, v) in zip(dat_items[0::2], dat_items[1::2])}
        print(' -dict assignment', dat_group, dat_items)

single assignment USB_SMSC_NXT {'IOSTANDARD': 'LVCMOS18'}
single assignment DDR_RAS_n {'PACKAGE_PIN': 'R5'}
single assignment MIO[53] {'PIO_DIRECTION': 'BIDIR'}
 -dict assignment FMC1_LA_24_N {'PACKAGE_PIN': 'A8', 'IOSTANDARD': 'DIFF_HSTL_II_25'}


In [1]:
from sys import argv
from xdc_parser import XdcParser

# Marble

In [2]:
p = XdcParser('/home/michael/fpga_wsp/bedrock/board_support/marble/Marble.xdc')

In [6]:
p.getGroup('eth', (
    ('rst_n', r'PHY_RSTn'),
    ('rx_dv', r'RGMII_RX_DV'),
    ('rx_data', r'RGMII_RXD\d'),
    ('tx_en', r'RGMII_TX_EN'),
    ('tx_data', r'RGMII_RXD\d')
))

    ("eth", 0,
        Subsignal("rst_n", Pins("B9"), IOStandard("LVCMOS25")),
        Subsignal("rx_dv", Pins("J11"), IOStandard("LVCMOS25")),
        Subsignal("rx_data", Pins("J10 J8 H8 H9"), IOStandard("LVCMOS25")),
        Subsignal("tx_en", Pins("C9"), IOStandard("LVCMOS25")),
        Subsignal("tx_data", Pins("J10 J8 H8 H9"), IOStandard("LVCMOS25")),
    ),


In [8]:
p.getGroup('clk20', (
    ('p', 'CLK20_VCXO'),
))

p.getGroup('clk125', (
    ('p', 'DDR_REF_CLK_C_P'),
    ('n', 'DDR_REF_CLK_C_N'),
))

for i in range(4):
    p.getGroup(f'clkmgt{i}', (
        ('p', f'MGT_CLK_{i}_P'),
        ('n', f'MGT_CLK_{i}_N'),
    ))

    ("clk20", 0,
        Subsignal("p", Pins("W11"), IOStandard("LVCMOS15")),
    ),
    ("clk125", 0,
        Subsignal("p", Pins("AC9"), IOStandard("DIFF_SSTL15")),
        Subsignal("n", Pins("AD9"), IOStandard("DIFF_SSTL15")),
    ),
    ("clkmgt0", 0,
        Subsignal("p", Pins("D6"), IOStandard("DIFF_SSTL15")),
        Subsignal("n", Pins("D5"), IOStandard("DIFF_SSTL15")),
    ),
    ("clkmgt1", 0,
        Subsignal("p", Pins("F6"), IOStandard("DIFF_SSTL15")),
        Subsignal("n", Pins("F5"), IOStandard("DIFF_SSTL15")),
    ),
    ("clkmgt2", 0,
        Subsignal("p", Pins("H6"), IOStandard("DIFF_SSTL15")),
        Subsignal("n", Pins("H5"), IOStandard("DIFF_SSTL15")),
    ),
    ("clkmgt3", 0,
        Subsignal("p", Pins("K6"), IOStandard("DIFF_SSTL15")),
        Subsignal("n", Pins("K5"), IOStandard("DIFF_SSTL15")),
    ),


In [9]:
p.getGpios('user_led', r'LD1[67]')

    ("user_led", 0, Pins("Y13"), IOStandard("LVCMOS15")),
    ("user_led", 1, Pins("V12"), IOStandard("LVCMOS15")),


In [10]:
p.getGroup('serial', (
    ('rx', 'FPGA_RxD'),
    ('rts', 'FPGA_RTS'),
    ('tx', 'FPGA_TxD'),
))

    ("serial", 0,
        Subsignal("rx", Pins("K15"), IOStandard("LVCMOS25")),
        Subsignal("rts", Pins("M16"), IOStandard("LVCMOS25")),
        Subsignal("tx", Pins("C16"), IOStandard("LVCMOS25")),
    ),


In [11]:
    p.getGroup('ddram', (
        ('a', r'DDR3_A\d+', 16),
        ('ba', r'DDR3_BA\d', 3),
        ('ras_n', 'DDR3_RAS_N'),
        ('cas_n', 'DDR3_CAS_N'),
        ('we_n', 'DDR3_WE_N'),
        ('cs_n', r'DDR3_CS_N'),
        ('dm', r'DDR3_DM\d', 8),
        ('dq', r'DDR3_DQ\d+', 64),
        ('dqs_p', r'DDR3_DQS\d_P', 8),
        ('dqs_n', r'DDR3_DQS\d_N', 8),
        ('clk_p', r'DDR3_CK_P'),
        ('clk_n', r'DDR3_CK_N'),
        ('cke', r'DDR3_CKE'),
        ('odt', r'DDR3_ODT'),
        ('reset_n', 'DDR3_RST_N'),
    ))

    ("ddram", 0,
        Subsignal("a", Pins("AC8 AB10 AA9 AA10 AD10 AC12 AB11 AC11 AF13 AE13 AE10 AD11 AA12 AE8 AB12 AD13"), IOStandard("SSTL15")),
        Subsignal("ba", Pins("AF10 AD8 AC13"), IOStandard("SSTL15")),
        Subsignal("ras_n", Pins("AB7"), IOStandard("SSTL15")),
        Subsignal("cas_n", Pins("AF8"), IOStandard("SSTL15")),
        Subsignal("we_n", Pins("AF9"), IOStandard("SSTL15")),
        Subsignal("cs_n", Pins("AC7"), IOStandard("SSTL15")),
        Subsignal("dm", Pins("AF17 W15 AC19 AA15 AC3 AD4 W1 U7"), IOStandard("SSTL15")),
        Subsignal("dq", Pins("AF20 AF19 AE17 AE15 AD16 AD15 AF15 AF14 V17 Y17 V18 V19 V16 W16 V14 W14 AA20 AD19 AB17 AC17 AA19 AB19 AD18 AC18 AA18 AB16 AA14 AD14 AB15 AA17 AC14 AB14 AD6 AB6 Y6 AC4 AC6 AB4 AA4 Y5 AF2 AE2 AE1 AD1 AE5 AE6 AF3 AE3 AA3 AC2 V2 V1 AB2 Y3 Y2 Y1 W3 V4 U2 U1 V6 V3 U6 U5"), IOStandard("SSTL15")),
        Subsignal("dqs_p", Pins("AE18 W18 AD20 Y15 AA5 AF5 AB1 W6"), IOStandard("DIFF_SSTL15")),
        Subsignal("dqs_n

In [6]:
p.getConnector('FMC1', (
    ('LA_', 'LA'),
))

    ("FMC1", {
        "CLK0_M2C_N": "E17",
        "CLK0_M2C_P": "F17",
        "CLK1_M2C_N": "D18",
        "CLK1_M2C_P": "E18",
        "LA0_N": "H18",
        "LA0_P": "H17",
        "LA1_N": "F18",
        "LA1_P": "G17",
        "LA2_N": "J20",
        "LA2_P": "K20",
        "LA3_N": "L18",
        "LA3_P": "M17",
        "LA4_N": "G20",
        "LA4_P": "H19",
        "LA5_N": "E20",
        "LA5_P": "F19",
        "LA6_N": "L20",
        "LA6_P": "L19",
        "LA7_N": "D20",
        "LA7_P": "D19",
        "LA8_N": "F20",
        "LA8_P": "G19",
        "LA9_N": "J19",
        "LA9_P": "J18",
        "LA10_N": "G16",
        "LA10_P": "H16",
        "LA11_N": "K18",
        "LA11_P": "L17",
        "LA12_N": "F15",
        "LA12_P": "G15",
        "LA13_N": "D16",
        "LA13_P": "D15",
        "LA14_N": "E16",
        "LA14_P": "E15",
        "LA15_N": "J16",
        "LA15_P": "J15",
        "LA16_N": "K17",
        "LA16_P": "K16",
        "LA17_N": "D10",
        "LA17_

In [7]:
p.getConnector('FMC2_', (
    ('LA_', 'LA'),
))

    ("FMC2", {
        "CLK0_M2C_N": "AA24",
        "CLK0_M2C_P": "Y23",
        "CLK1_M2C_N": "E23",
        "CLK1_M2C_P": "F22",
        "HA00_CC_N": "N22",
        "HA00_CC_P": "N21",
        "HA01_CC_N": "P21",
        "HA01_CC_P": "R21",
        "HA02_N": "U20",
        "HA02_P": "U19",
        "HA03_N": "T19",
        "HA03_P": "T18",
        "HA04_N": "R17",
        "HA04_P": "R16",
        "HA05_N": "N17",
        "HA05_P": "P16",
        "HA06_N": "P18",
        "HA06_P": "R18",
        "HA07_N": "T25",
        "HA07_P": "T24",
        "HA08_N": "T23",
        "HA08_P": "T22",
        "HA09_N": "T17",
        "HA09_P": "U17",
        "HA10_N": "K26",
        "HA10_P": "K25",
        "HA11_N": "M19",
        "HA11_P": "N18",
        "HA12_N": "L24",
        "HA12_P": "M24",
        "HA13_N": "R20",
        "HA13_P": "T20",
        "HA14_N": "P20",
        "HA14_P": "P19",
        "HA15_N": "P25",
        "HA15_P": "R25",
        "HA16_N": "P26",
        "HA16_P": "R26",
      

In [8]:
p.getConnector('Pmod1')

    ("Pmod1", {
        "0": "C24",
        "1": "C22",
        "2": "L23",
        "3": "D21",
        "4": "K21",
        "5": "C18",
        "6": "C19",
        "7": "C17",
    }),


In [12]:
for i in (1, 2):
    p.getConnector(f'Pmod{i}', noKeys=True, outName=f'pmod{i}')

    ("pmod1", "C24 C22 L23 D21 K21 C18 C19 C17"),
    ("pmod2", "AE7 V7 Y7 AF7 V8 AA8 Y8 W9"),


# VC707

In [3]:
p.getGroup('eth', (
    ('rst_n', 'PHY_RESET_LS'),
    ('int_n', 'PHY_INT_LS'),
    ('mdio', 'PHY_MDIO_LS'),
    ('mdc', 'PHY_MDC_LS'),
    ('rx_p', 'SGMII_RX_P'),
    ('rx_n', 'SGMII_RX_N'),
    ('tx_p', 'SGMII_TX_P'),
    ('tx_n', 'SGMII_TX_N'),
))
p.getGroup('sgmii_clock', (
    ('p', 'SGMIICLK_Q0_P'),
    ('n', 'SGMIICLK_Q0_N'),
))

    ("eth", 0,
        Subsignal("rst_n", Pins("")),
        Subsignal("int_n", Pins("")),
        Subsignal("mdio", Pins("")),
        Subsignal("mdc", Pins("")),
        Subsignal("rx_p", Pins("")),
        Subsignal("rx_n", Pins("")),
        Subsignal("tx_p", Pins("")),
        Subsignal("tx_n", Pins("")),
    ),
    ("sgmii_clock", 0,
        Subsignal("p", Pins("")),
        Subsignal("n", Pins("")),
    ),


In [339]:
for i in [1, 2, 4, 8]:
    p.getGroup('pcie_x{:}'.format(i), (
        ('rst_n', 'PCIE_PERST_.*'),
        ('clk_p', 'PCIE_CLK_QO_P'),
        ('clk_n', 'PCIE_CLK_QO_N'),
        ('rx_p', 'PCIE_RX[0-7]_P', i),
        ('rx_n', 'PCIE_RX[0-7]_N', i),
        ('tx_p', 'PCIE_TX[0-7]_P', i),
        ('tx_n', 'PCIE_TX[0-7]_N', i),
    ))

    ("pcie_x1", 0,
        Subsignal("rst_n", Pins("AV35"), IOStandard("LVCMOS18")),
        Subsignal("clk_p", Pins("AB8")),
        Subsignal("clk_n", Pins("AB7")),
        Subsignal("rx_p", Pins("Y4")),
        Subsignal("rx_n", Pins("Y3")),
        Subsignal("tx_p", Pins("W2")),
        Subsignal("tx_n", Pins("W1")),
    ),
    ("pcie_x2", 0,
        Subsignal("rst_n", Pins("AV35"), IOStandard("LVCMOS18")),
        Subsignal("clk_p", Pins("AB8")),
        Subsignal("clk_n", Pins("AB7")),
        Subsignal("rx_p", Pins("Y4 AA6")),
        Subsignal("rx_n", Pins("Y3 AA5")),
        Subsignal("tx_p", Pins("W2 AA2")),
        Subsignal("tx_n", Pins("W1 AA1")),
    ),
    ("pcie_x4", 0,
        Subsignal("rst_n", Pins("AV35"), IOStandard("LVCMOS18")),
        Subsignal("clk_p", Pins("AB8")),
        Subsignal("clk_n", Pins("AB7")),
        Subsignal("rx_p", Pins("Y4 AA6 AB4 AC6")),
        Subsignal("rx_n", Pins("Y3 AA5 AB3 AC5")),
        Subsignal("tx_p", Pins("W2 AA2 AC2 AE2")),
    

In [308]:
p.getGroup('clk200', (
    ('p', 'SYSCLK_P'),
    ('n', 'SYSCLK_N'),
))
p.getGpios('user_sma_clock_p', 'USER_SMA_CLOCK_P')
p.getGpios('user_sma_clock_n', 'USER_SMA_CLOCK_N')
p.getGroup('user_sma_clock', (
    ('p', 'USER_SMA_CLOCK_P'),
    ('n', 'USER_SMA_CLOCK_N'),
))
p.getGpios('user_clock_p', 'USER_CLOCK_P')
p.getGpios('user_clock_n', 'USER_CLOCK_N')
p.getGroup('user_sma_mgt_tx', (
    ('p', 'SMA_MGT_TX_P'),
    ('n', 'SMA_MGT_TX_N'),
))
p.getGroup('user_sma_mgt_rx', (
    ('p', 'SMA_MGT_RX_P'),
    ('n', 'SMA_MGT_RX_N'),
))
p.getGroup('user_sma_mgt_refclk', (
    ('p', 'SMA_MGT_REFCLK_P'),
    ('n', 'SMA_MGT_REFCLK_N'),
))
p.getGroup('si5324', (
    ('rst_n', 'SI5324_RST_LS'),
    ('int', 'SI5324_INT_ALM_LS'),
))
p.getGroup('si5324_clkin', (
    ('p', 'REC_CLOCK_C_P'),
    ('n', 'REC_CLOCK_C_N'),
))

("clk200", 0,
    Subsignal("p", Pins("E19"), IOStandard("LVDS")),
    Subsignal("n", Pins("E18"), IOStandard("LVDS")),
),
("user_sma_clock_p", 0, Pins("AJ32"), IOStandard("LVCMOS18")),
("user_sma_clock_n", 0, Pins("AK32"), IOStandard("LVCMOS18")),
("user_sma_clock", 0,
    Subsignal("p", Pins("AJ32"), IOStandard("LVCMOS18")),
    Subsignal("n", Pins("AK32"), IOStandard("LVCMOS18")),
),
("user_clock_p", 0, Pins("AK34"), IOStandard("LVDS")),
("user_clock_n", 0, Pins("AL34"), IOStandard("LVDS")),
("user_sma_mgt_tx", 0,
    Subsignal("p", Pins("AP4")),
    Subsignal("n", Pins("AP3")),
),
("user_sma_mgt_rx", 0,
    Subsignal("p", Pins("AN6")),
    Subsignal("n", Pins("AN5")),
),
("user_sma_mgt_refclk", 0,
    Subsignal("p", Pins("AK8")),
    Subsignal("n", Pins("AK7")),
),
("si5324", 0,
    Subsignal("rst_n", Pins("AT36"), IOStandard("LVCMOS18")),
    Subsignal("int", Pins("AU34"), IOStandard("LVCMOS18")),
),
("si5324_clkin", 0,
    Subsignal("p", Pins("AW32"), IOStandard("LVCMOS18")),
   

In [309]:
p.getGpios('cpu_reset', 'CPU_RESET')
p.getGpios('user_led', 'GPIO_LED_[0-9]_LS')
p.getGpios('user_dip_btn', 'GPIO_DIP_SW[0-9]')
p.getGpios('user_btn_c', 'GPIO_SW_C')
p.getGpios('user_btn_n', 'GPIO_SW_N')
p.getGpios('user_btn_e', 'GPIO_SW_E')
p.getGpios('user_btn_s', 'GPIO_SW_S')
p.getGpios('user_btn_w', 'GPIO_SW_W')
p.getGroup('rotary', (
    ('a', 'ROTARY_INCA'),
    ('b', 'ROTARY_INCB'),
    ('push', 'ROTARY_PUSH'),
))
p.getGpios('user_sma_gpio_p', 'USER_SMA_GPIO_P')
p.getGpios('user_sma_gpio_n', 'USER_SMA_GPIO_N')
p.getGroup('lcd', (
    ('db', 'LCD_DB[4-7]_LS'),
    ('rs', 'LCD_RS_LS'),
    ('rw', 'LCD_RW_LS'),
    ('e', 'LCD_E_LS'),
))
p.getGroup('i2c', (
    ('scl', 'IIC_SCL_MAIN_LS'),
    ('sda', 'IIC_SDA_MAIN_LS'),
))
p.getGpios('i2c_mux_reset', 'IIC_MUX_RESET_B_LS')
p.getConnector("XADC", nameReplace=(('N_R', '_N'),('P_R', '_P')))
p.getGroup('serial', (
    ('rx', 'USB_UART_RX'),
    ('rts', 'USB_UART_RTS'),
    ('tx', 'USB_UART_TX'),
    ('cts', 'USB_UART_CTS'),
))

("cpu_reset", 0, Pins("AV40"), IOStandard("LVCMOS18")),
("user_led", 0, Pins("AM39"), IOStandard("LVCMOS18")),
("user_led", 1, Pins("AN39"), IOStandard("LVCMOS18")),
("user_led", 2, Pins("AR37"), IOStandard("LVCMOS18")),
("user_led", 3, Pins("AT37"), IOStandard("LVCMOS18")),
("user_led", 4, Pins("AR35"), IOStandard("LVCMOS18")),
("user_led", 5, Pins("AP41"), IOStandard("LVCMOS18")),
("user_led", 6, Pins("AP42"), IOStandard("LVCMOS18")),
("user_led", 7, Pins("AU39"), IOStandard("LVCMOS18")),
("user_dip_btn", 0, Pins("AV30"), IOStandard("LVCMOS18")),
("user_dip_btn", 1, Pins("AY33"), IOStandard("LVCMOS18")),
("user_dip_btn", 2, Pins("BA31"), IOStandard("LVCMOS18")),
("user_dip_btn", 3, Pins("BA32"), IOStandard("LVCMOS18")),
("user_dip_btn", 4, Pins("AW30"), IOStandard("LVCMOS18")),
("user_dip_btn", 5, Pins("AY30"), IOStandard("LVCMOS18")),
("user_dip_btn", 6, Pins("BA30"), IOStandard("LVCMOS18")),
("user_dip_btn", 7, Pins("BB31"), IOStandard("LVCMOS18")),
("user_btn_c", 0, Pins("AV39"), 

In [310]:
p.getGroup('hdmi', (
    ('d', 'HDMI_R_D\d+', 36),
    ('de', 'HDMI_R_DE'),
    ('clk', 'HDMI_R_CLK'),
    ('vsync', 'HDMI_R_VSYNC'),
    ('hsync', 'HDMI_R_HSYNC'),
    ('int', 'HDMI_INT'),
    ('spdif', 'HDMI_R_SPDIF'),
    ('spdif_out', 'HDMI_SPDIF_OUT_LS')
))

("hdmi", 0,
    Subsignal("d", Pins("AM22 AL22 AJ20 AJ21 AM21 AL21 AK22 AJ22 AL20 AK20 AK23 AJ23 AN21 AP22 AP23 AN23 AM23 AN24 AY24 BB22 BA22 BA25 AY25 AY22 AY23 AV24 AU24 AW21 AV21 AT24 AR24 AU21 AT21 AW22 AW23 AV23"), IOStandard("LVCMOS18")),
    Subsignal("de", Pins("AP21"), IOStandard("LVCMOS18")),
    Subsignal("clk", Pins("AU23"), IOStandard("LVCMOS18")),
    Subsignal("vsync", Pins("AT22"), IOStandard("LVCMOS18")),
    Subsignal("hsync", Pins("AU22"), IOStandard("LVCMOS18")),
    Subsignal("int", Pins("AM24"), IOStandard("LVCMOS18")),
    Subsignal("spdif", Pins("AR23"), IOStandard("LVCMOS18")),
    Subsignal("spdif_out", Pins("AR22"), IOStandard("LVCMOS18")),
),


In [326]:
for n, i in (('ddram', 1), ('ddram_dual_rank', 2)):
    p.getGroup(n, (
        ('a', 'DDR3_A\d+', 16),
        ('ba', 'DDR3_BA\d', 3),
        ('ras_n', 'DDR3_RAS_B'),
        ('cas_n', 'DDR3_CAS_B'),
        ('we_n', 'DDR3_WE_B'),
        ('cs_n', 'DDR3_S\d_B', i), # hope this is right ???
        ('dm', 'DDR3_DM\d', 8),
        ('dq', 'DDR3_D\d+', 64),
        ('dqs_p', 'DDR3_DQS\d_P', 8),
        ('dqs_n', 'DDR3_DQS\d_N', 8),
        ('clk_p', 'DDR3_CLK\d_P', i),
        ('clk_n', 'DDR3_CLK\d_N', i),
        ('cke', 'DDR3_CKE\d', i),
        ('odt', 'DDR3_ODT\d', i),
        ('reset_n', 'DDR3_RESET_B'),   
    ))

("ddram", 0,
    Subsignal("a", Pins("A20 B19 C20 A19 A17 A16 D20 C18 D17 C19 B21 B17 A15 A21 F17 E17"), IOStandard("SSTL15")),
    Subsignal("ba", Pins("D21 C21 D18"), IOStandard("SSTL15")),
    Subsignal("ras_n", Pins("E20"), IOStandard("SSTL15")),
    Subsignal("cas_n", Pins("K17"), IOStandard("SSTL15")),
    Subsignal("we_n", Pins("F20"), IOStandard("SSTL15")),
    Subsignal("cs_n", Pins("J17"), IOStandard("SSTL15")),
    Subsignal("dm", Pins("M13 K15 F12 A14 C23 D25 C31 F31"), IOStandard("SSTL15")),
    Subsignal("dq", Pins("N14 N13 L14 M14 M12 N15 M11 L12 K14 K13 H13 J13 L16 L15 H14 J15 E15 E13 F15 E14 G13 G12 F14 G14 B14 C13 B16 D15 D13 E12 C16 D16 A24 B23 B27 B26 A22 B22 A25 C24 E24 D23 D26 C25 E23 D22 F22 E22 A30 D27 A29 C28 D28 B31 A31 A32 E30 F29 F30 F27 C30 E29 F26 D30"), IOStandard("SSTL15")),
    Subsignal("dqs_p", Pins("N16 K12 H16 C15 A26 F25 B28 E27"), IOStandard("DIFF_SSTL15")),
    Subsignal("dqs_n", Pins("M16 J12 G16 C14 A27 E25 B29 E28"), IOStandard("DIFF_SSTL15"))

In [335]:
p.getGroup('sfp', (
    ('txp', 'SFP_TX_P'),
    ('txn', 'SFP_TX_N'),
    ('rxp', 'SFP_RX_P'),
    ('rxn', 'SFP_RX_N')
))
p.getGroup('sfp_tx', (
    ('txp', 'SFP_TX_P'),
    ('txn', 'SFP_TX_N'),
))
p.getGroup('sfp_rx', (
    ('rxp', 'SFP_RX_P'),
    ('rxn', 'SFP_RX_N')
))
p.getGpios('sfp_tx_disable_n', 'SFP_TX_DISABLE')
p.getGpios('sfp_rx_los', 'SFP_LOS_LS')

("sfp", 0,
    Subsignal("txp", Pins("AM4")),
    Subsignal("txn", Pins("AM3")),
    Subsignal("rxp", Pins("AL6")),
    Subsignal("rxn", Pins("AL5")),
),
("sfp_tx", 0,
    Subsignal("txp", Pins("AM4")),
    Subsignal("txn", Pins("AM3")),
),
("sfp_rx", 0,
    Subsignal("rxp", Pins("AL6")),
    Subsignal("rxn", Pins("AL5")),
),
("sfp_tx_disable_n", 0, Pins("AP33"), IOStandard("LVCMOS18")),
("sfp_rx_los", 0, Pins("BB38"), IOStandard("LVCMOS18")),


In [327]:
p.props["SDIO_DAT3_LS"] = p.props["SDIO_CD_DAT3_LS"]
p.keys = natsorted(p.props.keys())
p.getGroup('mmc', (
    ('clk', 'SDIO_CLK_LS'),
    ('cmd', 'SDIO_CMD_LS'),
    ('det', 'SDIO_SDDET'),
    ('wp', 'SDIO_SDWP'),
    ('dat', 'SDIO_DAT[0-3]_LS', 4),
))

("mmc", 0,
    Subsignal("clk", Pins("AN30"), IOStandard("LVCMOS18")),
    Subsignal("cmd", Pins("AP30"), IOStandard("LVCMOS18")),
    Subsignal("det", Pins("AP32"), IOStandard("LVCMOS18")),
    Subsignal("wp", Pins("AR32"), IOStandard("LVCMOS18")),
    Subsignal("dat", Pins("AR30 AU31 AV31 AT30"), IOStandard("LVCMOS18")),
),


In [330]:
p.getGpios('vadj_on_b', 'FMC_VADJ_ON_B_LS')
p.getConnector('FMC1_HPC')

("vadj_on_b", 0, Pins("AH35"), IOStandard("LVCMOS18")),
("FMC1_HPC", {
    "CLK0_M2C_N": "L40",
    "CLK0_M2C_P": "L39",
    "CLK1_M2C_N": "M31",
    "CLK1_M2C_P": "N30",
    "DP0_C2M_N": "E1",
    "DP0_C2M_P": "E2",
    "DP0_M2C_N": "D7",
    "DP0_M2C_P": "D8",
    "DP1_C2M_N": "D3",
    "DP1_C2M_P": "D4",
    "DP1_M2C_N": "C5",
    "DP1_M2C_P": "C6",
    "DP2_C2M_N": "C1",
    "DP2_C2M_P": "C2",
    "DP2_M2C_N": "B7",
    "DP2_M2C_P": "B8",
    "DP3_C2M_N": "B3",
    "DP3_C2M_P": "B4",
    "DP3_M2C_N": "A5",
    "DP3_M2C_P": "A6",
    "DP4_C2M_N": "J1",
    "DP4_C2M_P": "J2",
    "DP4_M2C_N": "H7",
    "DP4_M2C_P": "H8",
    "DP5_C2M_N": "H3",
    "DP5_C2M_P": "H4",
    "DP5_M2C_N": "G5",
    "DP5_M2C_P": "G6",
    "DP6_C2M_N": "G1",
    "DP6_C2M_P": "G2",
    "DP6_M2C_N": "F7",
    "DP6_M2C_P": "F8",
    "DP7_C2M_N": "F3",
    "DP7_C2M_P": "F4",
    "DP7_M2C_N": "E5",
    "DP7_M2C_P": "E6",
    "GBTCLK0_M2C_C_N": "A9",
    "GBTCLK0_M2C_C_P": "A10",
    "GBTCLK1_M2C_C_N": "E9",
    "

In [128]:
p.getConnector('FMC2_HPC')

("FMC2_HPC", {
    "CLK0_M2C_N": "AF40",
    "CLK0_M2C_P": "AF39",
    "CLK1_M2C_N": "T39",
    "CLK1_M2C_P": "U39",
    "DP0_C2M_N": "N1",
    "DP0_C2M_P": "N2",
    "DP0_M2C_N": "P7",
    "DP0_M2C_P": "P8",
    "DP1_C2M_N": "M3",
    "DP1_C2M_P": "M4",
    "DP1_M2C_N": "N5",
    "DP1_M2C_P": "N6",
    "DP2_C2M_N": "L1",
    "DP2_C2M_P": "L2",
    "DP2_M2C_N": "L5",
    "DP2_M2C_P": "L6",
    "DP3_C2M_N": "K3",
    "DP3_C2M_P": "K4",
    "DP3_M2C_N": "J5",
    "DP3_M2C_P": "J6",
    "DP4_C2M_N": "U1",
    "DP4_C2M_P": "U2",
    "DP4_M2C_N": "W5",
    "DP4_M2C_P": "W6",
    "DP5_C2M_N": "T3",
    "DP5_C2M_P": "T4",
    "DP5_M2C_N": "V3",
    "DP5_M2C_P": "V4",
    "DP6_C2M_N": "R1",
    "DP6_C2M_P": "R2",
    "DP6_M2C_N": "U5",
    "DP6_M2C_P": "U6",
    "DP7_C2M_N": "P3",
    "DP7_C2M_P": "P4",
    "DP7_M2C_N": "R5",
    "DP7_M2C_P": "R6",
    "GBTCLK0_M2C_C_N": "K7",
    "GBTCLK0_M2C_C_P": "K8",
    "GBTCLK1_M2C_C_N": "T7",
    "GBTCLK1_M2C_C_P": "T8",
    "HA00_CC_N": "AC33",
    "H